In [1]:
import pickle
import pandas as pd
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing

from prophet import Prophet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.svm import SVR
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np

# Load data
dfd = pd.read_csv("daily.csv")
dfd['date'] = pd.to_datetime(dfd['date'])
dfd.set_index('date', inplace=True)

# Perform differencing
dfd_diff = dfd.diff().dropna()

# Train-test split
train_size = int(len(dfd_diff) * 0.8)
df_train, df_test = dfd_diff.iloc[:train_size], dfd_diff.iloc[train_size:]

# Normalize the data
scaler = StandardScaler()
train_data_scaled = scaler.fit_transform(df_train[['cycle']])
test_data_scaled = scaler.transform(df_test[['cycle']])

# Define and fit ARIMA model
arima_model = ARIMA(train_data_scaled, order=(2, 1, 0))
arima_model_fit = arima_model.fit()
with open('arima_model.pkl', 'wb') as f:
    pickle.dump(arima_model_fit, f)

# Define and fit SARIMA model
sarima_model = SARIMAX(df_train['cycle'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
sarima_model_fit = sarima_model.fit()
with open('sarima_model.pkl', 'wb') as f:
    pickle.dump(sarima_model_fit, f)

# Define and fit ETS model
ets_model = ExponentialSmoothing(df_train['cycle'])
ets_model_fit = ets_model.fit()
with open('ets_model.pkl', 'wb') as f:
    pickle.dump(ets_model_fit, f)

# Define and fit Prophet model
prophet_model = Prophet()
prophet_model.fit(pd.DataFrame({'ds': df_train.index, 'y': df_train['cycle'].values}))
with open('prophet_model.pkl', 'wb') as f:
    pickle.dump(prophet_model, f)

# Features and target for ANN model
X_train, X_test, y_train, y_test = train_test_split(df_train[['cycle', 'trend']], df_train['cycle'], test_size=0.2, random_state=42)

# Scale data for ANN model
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define and fit ANN model
ann_model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)
])
ann_model.compile(optimizer='adam', loss='mean_squared_error')
ann_model.fit(X_train_scaled, y_train, epochs=100, validation_split=0.2, verbose=0)
ann_model.save('ann_model.h5')

# Define and fit SVR model
svr_model = SVR(kernel='rbf')
svr_model.fit(X_train_scaled, y_train)
with open('svr_model.pkl', 'wb') as f:
    pickle.dump(svr_model, f)

# Define and fit LSTM model
X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))
lstm_model = Sequential()
lstm_model.add(LSTM(50, activation='relu', input_shape=(1, X_train_scaled.shape[1])))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mse')
lstm_model.fit(X_train_lstm, y_train, epochs=50, batch_size=16, verbose=0)
lstm_model.save('lstm_model.h5')

# Hybrid model doesn't need to be stored separately as it combines ARIMA and ANN

# Store scaler for future use
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)


/Users/nibrasaamir/anaconda3/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/nibrasaamir/anaconda3/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -2.97819D+00    |proj g|=  2.06248D+02

At iterate    5    f= -3.21532D+00    |proj g|=  3.04313D+00

At iterate   10    f= -3.51622D+00    |proj g|=  4.88990D+01

At iterate   15    f= -3.59075D+00    |proj g|=  1.39437D+00

At iterate   20    f= -3.60606D+00    |proj g|=  3.67714D+01

At iterate   25    f= -3.69070D+00    |proj g|=  6.27484D+00

At iterate   30    f= -3.69119D+00    |proj g|=  1.04174D-01

At iterate   35    f= -3.69206D+00    |proj g|=  3.00770D+00

At iterate   40    f= -3.69900D+00    |proj g|=  9.17810D-01

At iterate   45    f= -3.72256D+00    |proj g|=  9.98673D+00

At iterate   50    f= -3.77576D+00    |proj g|=  8.67469D-01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cau

/Users/nibrasaamir/anaconda3/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/nibrasaamir/anaconda3/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
16:03:27 - cmdstanpy - INFO - Chain [1] start processing
16:03:27 - cmdstanpy - INFO - Chain [1] done processing
/Users/nibrasaamir/anaconda3/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/nibrasaamir/anaconda3/lib/python3.10/site-packages/keras/src/layers/rnn

In [2]:
import pickle
import pandas as pd
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.arima.model import ARIMAResults
from statsmodels.tsa.statespace.sarimax import SARIMAXResults
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from prophet import Prophet
from tensorflow.keras.models import load_model
from sklearn.svm import SVR
import numpy as np

# Load data
dfd = pd.read_csv("daily.csv")
dfd['date'] = pd.to_datetime(dfd['date'])
dfd.set_index('date', inplace=True)

# Load scaler
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

# Perform differencing
dfd_diff = dfd.diff().dropna()

# Train-test split
train_size = int(len(dfd_diff) * 0.8)
df_train, df_test = dfd_diff.iloc[:train_size], dfd_diff.iloc[train_size:]

# Normalize the data
train_data_scaled = scaler.transform(df_train)
test_data_scaled = scaler.transform(df_test)

# Load ARIMA model
with open('arima_model.pkl', 'rb') as f:
    arima_model_fit = pickle.load(f)

# Load SARIMA model
with open('sarima_model.pkl', 'rb') as f:
    sarima_model_fit = pickle.load(f)

# Load ETS model
with open('ets_model.pkl', 'rb') as f:
    ets_model_fit = pickle.load(f)

# Load Prophet model
with open('prophet_model.pkl', 'rb') as f:
    prophet_model = pickle.load(f)

# Features and target for ANN model
X_train, X_test, y_train, y_test = train_test_split(df_train[['cycle', 'trend']], df_train['cycle'], test_size=0.2, random_state=42)

# Scale data for ANN model
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Load ANN model
ann_model = load_model('ann_model.h5')

# Load SVR model
with open('svr_model.pkl', 'rb') as f:
    svr_model = pickle.load(f)

# Load LSTM model
lstm_model = load_model('lstm_model.h5', custom_objects={'mse': 'mse'})




In [3]:
X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# Make predictions using the loaded models
arima_forecast = arima_model_fit.forecast(steps=len(test_data_scaled))
sarima_forecast = sarima_model_fit.forecast(steps=len(df_test))
ets_forecast = ets_model_fit.forecast(steps=len(df_test))
# Define future dates for Prophet model
future = prophet_model.make_future_dataframe(periods=len(df_test))

prophet_forecast = prophet_model.predict(future)
ann_predictions = ann_model.predict(X_test_scaled)
svr_predictions = svr_model.predict(X_test_scaled)
lstm_predictions = lstm_model.predict(X_test_lstm)

# Evaluate the models
arima_mae = mean_absolute_error(df_test['cycle'], arima_forecast)
sarima_mae = mean_absolute_error(df_test['cycle'], sarima_forecast)
ets_mae = mean_absolute_error(df_test['cycle'], ets_forecast)
prophet_mae = mean_absolute_error(df_test['cycle'], prophet_forecast['yhat'].tail(len(df_test)))
ann_mae = mean_absolute_error(y_test, ann_predictions)
svr_mae = mean_absolute_error(y_test, svr_predictions)
lstm_mae = mean_absolute_error(y_test, lstm_predictions)

# Print the MAE for each model
print("ARIMA MAE:", arima_mae)
print("SARIMA MAE:", sarima_mae)
print("ETS MAE:", ets_mae)
print("Prophet MAE:", prophet_mae)
print("ANN MAE:", ann_mae)
print("SVR MAE:", svr_mae)
print("LSTM MAE:", lstm_mae)


19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
ARIMA MAE: 0.16484474558982645
SARIMA MAE: 0.034207603047681265
ETS MAE: 0.03413388123847576
Prophet MAE: 0.00820042740368562
ANN MAE: 0.0007287237502430285
SVR MAE: 0.03718588640275117
LSTM MAE: 0.00013802498789367444


In [4]:
# another_script.py
import pickle
import pandas as pd
from sklearn.metrics import mean_squared_error
from hybridmodel import HybridModel

# Load the hybrid model instance
with open('hybrid_model_instance.pkl', 'rb') as f:
    hybridd_model = pickle.load(f)

# Load data again to ensure consistency
dfd = pd.read_csv("daily.csv")
dfd['date'] = pd.to_datetime(dfd['date'])
dfd.set_index('date', inplace=True)

# Perform differencing
dfd_diff = dfd.diff().dropna()

# Train-test split
train_size = int(len(dfd_diff) * 0.8)
df_train, df_test = dfd_diff.iloc[:train_size], dfd_diff.iloc[train_size:]

# Forecast using the loaded hybrid model instance
hybrid_forecast = hybridd_model.forecast(df_train['cycle'], df_test['cycle'])

# Calculate RMSE
hybrid_rmse = np.sqrt(mean_squared_error(df_test['cycle'], hybrid_forecast))
print(f"Hybrid RMSE: {hybrid_rmse}")


23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Hybrid RMSE: 0.046737402734844155
